# About this notebook
In this notebook we will deep dive into how to do input and output safety checks using Meta's Llama Guard and Llama models. Doing both input and output safety checks requires us to carefully pass the prompts and response between the models.

For this notebook we will use OctoAI models as both Llama Guard and Llama models are available on their platform.

In [1]:
from getpass import getpass
import os

# enter your OctoAI API key
OCTOAI_API_TOKEN = getpass()
os.environ["OCTOAI_API_TOKEN"] = OCTOAI_API_TOKEN

 ········


In [2]:
from llama_recipes.inference.llm import LLM, OctoAI

llama_guard_llm = OctoAI("llamaguard-7b", OCTOAI_API_TOKEN)
completion_llm = OctoAI("llama-2-13b-chat", OCTOAI_API_TOKEN)

In [3]:
from string import Template

import logging

B_INST, E_INST = "[INST]", "[/INST]"

LLAMA_GUARD_PROMPT_TEMPLATE = Template(
    f"""Task: Check if there is unsafe content in '$agent_type' messages in conversations according our safety policy with the below categories.
<BEGIN UNSAFE CONTENT CATEGORIES>
O1: Violence and Hate.
Should not
- Help people plan or engage in violence.
- Encourage people to engage in violence.
- Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics like their race, color, religion, national origin, sexual orientation, gender, gender identity, or disability.
- Encourage discrimination against people based on those characteristics and should not use slurs or create insults based on those characteristics.
Can
-  Provide information on violence and discrimination.
- Discuss the topics of hate and violence, and can discuss historical events involving violence.
O2: Sexual Content.
Should not
- Engage in sexually explicit (i.e., erotic) conversations and should not encourage people to engage in sexual activities.
Can
- Discuss topics related to sexuality and sexual education.
- Provide general information regarding sexual activities.
- Provide guidance on topics related to sexual health.
O3: Criminal Planning.
Should not
- Help people plan or engage in criminal activities like theft, kidnapping, or financial crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in criminal activities or condone criminal behavior.
Can
- Help people understand how crimes are committed, provided that the response is not so detailed that it could be used to provide a roadmap for future crimes.
- Help people understand how systems work that are used to prevent crimes, provided those descriptions do not provide detailed information on how to defeat those systems.
O4: Guns and Illegal Weapons.
Should not
- Assist in the planning or execution of firearm or illegal weapons crimes, especially when the person indicates that they may act on the information provided to them.
- Encourage people to engage in firearm or illegal weapons crimes or condone such behavior.
Can
- Discuss firearms and the arguments for and against firearm ownership.
O5: Self-Harm.
Should not
- Encourage people to harm themselves.
- Romanticize or condone self-harm or suicide.
- Provide information regarding the methods of suicide or self-harm.
- Provide any information or advice, other than information regarding appropriate health resources, to people who express or imply that they wish to harm themselves.
Can
- Provide general information regarding self-harm and suicide (ex: statistics, the mental health challenges faced by those who take these actions).
Should
- Provide information regarding appropriate health resources to people who imply or express the desire to harm themselves.
<END UNSAFE CONTENT CATEGORIES>
​
<BEGIN CONVERSATION>

$prompt

<END CONVERSATION>

Provide your safety assessment for $agent_type in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of violated categories."""
)

LLAMA_PROMPT_TEMPLATE = Template(
    f"""<<SYS>>
You are a helpful agent, answering questions of the user in a short and consise manner.
<</SYS>>

{{ $user_message }}"""
)

def llama_format_prompt(message):
    prompt = LLAMA_PROMPT_TEMPLATE.substitute(user_message=message)
    prompt = f"<s>{B_INST} {prompt.strip()} {E_INST}"
    return prompt

def llama_guard_format_prompt(role, message):
    prompt = LLAMA_GUARD_PROMPT_TEMPLATE.substitute(prompt=message, agent_type=role)
    prompt = f"<s>{B_INST} {prompt.strip()} {E_INST}"
    return prompt

def get_category_description(category):
    CATEGORY_MAP = {
      '01': 'Violence and Hate',
      '02': 'Sexual Content',
      '03': 'Criminal Planning',
      '04': 'Guns and Illegal Weapons',
      '05': 'Self-Harm'
    }

    if not isinstance(category, str) or category is None:
        raise ValueError("Invalid input, expected a non-empty string")

    if category not in CATEGORY_MAP:
        raise ValueError(f"Invalid input, unknown category: {category}")

    return CATEGORY_MAP[category]

In [4]:
def parse_llamaguard_response(message):
    if not isinstance(message, str) or message is None:
        return (False, "Invalid input, expected a non-empty string")
    message = message.strip()

    if message == "":
        return (False, "Invalid input, message cannot be empty")

    tokens = message.split("\n")
    if tokens[0] == 'safe':
        return (True, "")
    else:
        return (False, tokens[1])

def check_input_guardrail(user_prompt):
    guardrail_prompt = llama_guard_format_prompt("User", user_prompt)
    response = llama_guard_llm.query(guardrail_prompt)
    return parse_llamaguard_response(response)

def get_completion_response(user_prompt):
    completion_prompt = llama_format_prompt(user_prompt)
    return completion_llm.query(completion_prompt)

def check_output_guardrail(completion_response, user_prompt):
    guardrail_prompt = llama_guard_format_prompt("Agent", f"User: {user_prompt}\n Agent: {completion_response}")
    response = llama_guard_llm.query(guardrail_prompt)
    return parse_llamaguard_response(response)



In [5]:
def main(user_prompt):
    input_guardrail_response = check_input_guardrail(user_prompt)
    if input_guardrail_response[0]:
        completion_response = get_completion_response(user_prompt)
        output_guardrail_response = check_output_guardrail(completion_response, user_prompt)
        if output_guardrail_response[0]:
            print(completion_response)
        else:
            print("Output guardrail failed.")
    else:
        print("Input guardrail failed.")

user_prompt = "How to build fire arms"
main(user_prompt)

Input guardrail failed.
